In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
#Getting dates and gameid for each game in 2020/2021 English Premier League

url='https://api.sportsdata.io/v3/soccer/scores/json/CompetitionDetails/EPL'

response=requests.get(url, headers = {"Ocp-Apim-Subscription-Key": "7e69e12a9f714caebb8c178cf904e445"})

result=pd.json_normalize(response.json())
games = pd.DataFrame(result['Games'][0])#.flatten()#.columns

games_final = games[['GameId','AwayTeamId','HomeTeamId','Day','AwayTeamName','HomeTeamName']]

Days=games_final['Day'].unique().tolist()

Days=[element[0:10] for element in Days]

games_final['Day'] = pd.to_datetime(games_final['Day'])

TeamID = games_final['AwayTeamId'].unique().tolist()

Days=Days[:18]  ##### IMPORTANT: FOR DATES BEYOND THIS, EVENTS STILL HAVEN'T HAPPEN, SO CODE DOESN'T WORK

#Function that returns a dataframe of the games for each date

def daily_games(date):
    
    url='https://api.sportsdata.io/v3/soccer/odds/json/GameOddsByDate/'+date

    response=requests.get(url, headers = {"Ocp-Apim-Subscription-Key": "7e69e12a9f714caebb8c178cf904e445"})

    response=response.json()
    result=pd.DataFrame(response)

    #Selecting games only from EPL

    result=result[result['RoundId']==602].reset_index()

    
    #Selecting games which happened

    result=result[result['Status']=='Final']

    #Aggregated Odds per game 

    odds=result['PregameOdds']

    empty_list = []

    for i in range(len(odds)):
    
        try:
        
            empty_list.append(pd.DataFrame(odds[i]))
    
        except: 
        
            continue
        
    games_odds=pd.concat(empty_list)

    games=games_odds.groupby('GameId').agg({'HomeMoneyLine':'mean','AwayMoneyLine':'mean','DrawMoneyLine':'mean'})

    #Merging with initial table

    daily_games = pd.merge(left = result,
                                 right = games,
                                 how = 'inner', 
                                 left_on = "GameId", 
                                 right_on= "GameId")


    daily_games=daily_games[['GameId', 'RoundId','Day', 
                         'AwayTeamId', 'HomeTeamId', 'AwayTeamName', 'HomeTeamName','HomeTeamScore',
                         'AwayTeamScore', 'TotalScore','HomeMoneyLine', 'AwayMoneyLine', 'DrawMoneyLine']]

    daily_games[['Day']]=pd.to_datetime(daily_games['Day'])
    
    return daily_games

#Applying above function to all dates contained in the list previously obtained. 
#The result is a dataframe with all games, scores and bets for the 2020/2021 EPL

final_result=pd.concat(list(map(daily_games,Days)))

#Building a column for the actual winner, based on scores presented

def winner(row):
    if row['HomeTeamScore']>row['AwayTeamScore']:
        
        return row['HomeTeamName']
    
    elif row['HomeTeamScore']<row['AwayTeamScore']:
        
        return row['AwayTeamName']
    
    elif row['HomeTeamScore']==row['AwayTeamScore']:
        
        return 'Tie'

final_result['Winner']=final_result.apply(winner,axis=1)

#Building a column for the predicted winner, based on bets presented

def moneyline(row):
    minimum=min(row['HomeMoneyLine'], row['AwayMoneyLine'], row['DrawMoneyLine'])
    if row['HomeMoneyLine']==minimum:
        return row['HomeTeamName']
    elif row['AwayMoneyLine']==minimum:
        return row['AwayTeamName']
    elif row['DrawMoneyLine']==minimum:
        return 'Tie'
    
final_result['Moneyline']=final_result.apply(moneyline,axis=1)

#Rearraging the dataframe 

final_result.reset_index(inplace=True)

final_result.drop(['index'],axis=1)

final_result=final_result[['Day','HomeTeamName','AwayTeamName','Winner',
       'Moneyline']]

In [5]:
final_result

,Day,HomeTeamName,AwayTeamName,Winner,Moneyline
0,2020-09-12,Fulham FC,Arsenal FC,Arsenal FC,Arsenal FC
1,2020-09-12,Crystal Palace FC,Southampton FC,Crystal Palace FC,Southampton FC
2,2020-09-13,West Bromwich Albion FC,Leicester City FC,Leicester City FC,Leicester City FC
3,2020-09-13,Tottenham Hotspur FC,Everton FC,Everton FC,Tottenham Hotspur FC
4,2020-09-14,Sheffield United FC,Wolverhampton Wanderers FC,Tie,Wolverhampton Wanderers FC
5,2020-09-14,Brighton & Hove Albion FC,Chelsea FC,Chelsea FC,Chelsea FC
6,2020-09-19,Everton FC,West Bromwich Albion FC,Everton FC,Everton FC
7,2020-09-19,Leeds United FC,Fulham FC,Tie,Leeds United FC
8,2020-09-19,Manchester United FC,Crystal Palace FC,Crystal Palace FC,Manchester United FC
9,2020-09-19,Arsenal FC,West Ham United FC,Tie,Arsenal FC
